# ソルバとデプロイのスイッチについて

* [Is Plug-in Solver Sample-Efficient for Feature-based Reinforcement Learning?](https://arxiv.org/abs/2010.05673)
* [Online Sub-Sampling for Reinforcement Learning with General Function Approximation](https://arxiv.org/abs/2106.07203)
* [Provably Efficient Q-Learning with Low Switching Cost](https://arxiv.org/abs/1905.12849)


## 方策のスイッチ回数の話

* [Provably Efficient Q-Learning with Low Switching Cost](https://arxiv.org/abs/1905.12849)

設定：
* よくある有限ホライゾンMDPです．
* リグレット：$\operatorname{Regret}(K):=\sum_{k=1}^K\left[V_1^{\star}\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right]$

普通のRLの論文では方策の切り替え回数を気にしないですが，一般に実応用では方策を切り替えるのはあまり嬉しくありません（ChatGPTとかの性能がコロコロ変わると不安になりますね）

今回はデプロイする方策がなるべく変わらないようなアルゴリズムを設計することを目標にします．
そこで，次の「切り替えコスト」を導入します：

$$
n_{\text {switch }}\left(\pi, \pi^{\prime}\right):=\left|\left\{(h, x) \in[H] \times \mathcal{S}: \pi^h(x) \neq\left[\pi^{\prime}\right]^h(x)\right\}\right|
$$
これはつまり，$\pi$と$\pi'$の意思決定が異なる状態とステップの数を表してます．
特に方策$(\pi_1, \dots, \pi_K)$を吐き出すRLアルゴリズムでは，次のコストを定義します：

$$
N_{\text {switch }}:=\sum_{k=1}^{K-1} n_{\text {switch }}\left(\pi_k, \pi_{k+1}\right)
$$

明らかに任意のアルゴリズムについて$n_{\text {switch }} \leq HS$と$N_{\text {switch }}(\mathcal{A}) \leq H S(K-1)$が成立します．


### バンディットの場合

参考：
* [Finite-time Analysis of the Multiarmed Bandit Problem](https://link.springer.com/article/10.1023/A:1013689704352)

まずは簡単のためにバンディットについて考えましょう．
[普通のUCBアルゴリズム](BANDIT_UCB_regret_proof.ipynb)はUpper confidence boundはOptimal regretを達成しますが，Switching costについては特に何も言ってません．
一方で，次のUCB2アルゴリズムはもっと良いSwitching costを達成できます（また，元論文を見ればわかりますが，UCB２はリグレットの係数がタイトです）．

* 例えばアーム$a$のUCBが最大だとします．普通のUCBアルゴリズムは$a$のアームを一回引き，次のUCBを計算します．
* 一方で，UCB2は$a$のアームを複数回引きます．

具体的には次のアルゴリズムで進めます：

$N_j$をアーム$j$を引いた回数とします．
次の関数を定義します：
$$
\tau(N_j)=(1+\eta)^{N_j}
$$

1. $\bar{r}_j+b_{N_j}$を最大にするようなアーム$j$を選択します．ここで$\bar{r}_\cdot$はアームの報酬の推定値で，$b_N=O(\sqrt{\log T / \tau(N)})$はボーナスです．
2. アーム$j$を$\tau\left(N_j+1\right)-\tau\left(N_j\right)$回引きます

このとき，一定の$T$以降のUCB2のリグレットは$log(T)$でバウンド可能であり，さらにswitching costは$O\left(\frac{A \log (T / A)}{\eta}\right)$でバウンドできます．

---

**証明**

任意の$N \geq 1$に対して次が成立します：
$$
\tau(N) \leq(1+\eta)^N+1 \leq \tau(N-1)(1+\eta)+1
$$

すべてのアーム$j$について，$T \geq 1 / (2 \Delta_j^2)$として，

$$
\tau\left(\tilde{N}_j-1\right) \leq \frac{(1+4 \alpha) \ln \left(2 e n \Delta_j^2\right)}{2 \Delta_j^2}
$$

を満たすような最大の整数を$\tilde{N}_j$とします．明らかに$\tilde{N}_j \geq 1$です．

すると，
$$
\begin{aligned}
N_j(T) & \leq 1+\sum_{N \geq 1}(\tau(N)-\tau(N-1))\mathbb{I}\{\text { アーム } j \text { を } N \text {-th epoch で引く\} } \\
& \leq \tau\left(\tilde{r}_j\right)+\sum_{r>\tilde{r}_j}(\tau(r)-\tau(r-1)) \text { \{machine } j \text { finishes its } r \text {-th epoch\} }
\end{aligned}
$$


### RLアルゴリズムへの拡張

UCB2のアイデアをRLでも使います．

２つQ関数を用意しましょう．
* $\widetilde{Q}$：毎エピソードで更新されます
* $Q$：行動の選択に使います．たまに更新されます．

$$
\tau(r)=\left\lceil(1+\eta)^r\right\rceil, \quad r=1,2, \ldots
$$

とします．また，
$$
\left\{t_n\right\}_{n \geq 1}=\left\{1,2, \ldots, \tau\left(r_{\star}\right)\right\} \cup\left\{\tau\left(r_{\star}+1\right), \tau\left(r_{\star}+2\right), \ldots\right\},
$$
とし，$(\eta, r_\star)$はアルゴリズムの入力とします．
任意の$t \in {1, 2, \dots}$に対して，

$$
\tau_{\text {last }}(t):=\max \left\{t_n: t_n \leq t\right\} \quad \text { and } \quad \alpha_t=\frac{H+1}{H+t}
$$

とします．そして，

1. もし$t \leq \tau (r_\star)$ならば，$Q_h\left(x_h, a_h\right) \leftarrow \widetilde{Q}_h\left(x_h, a_h\right)$の更新を毎回行います
2. もし$t > \tau (r_\star)$ならば，$t=\tau(r)=\left\lceil(1+\eta)^r\right\rceil$ for some $r>r_{\star}$が成り立つときだけ更新します．

つまり，ある程度の更新回数以降は，UCB2と同じ頻度で方策の更新を行います．
ちなみにこの更新方法はPACの保証もできます．



## ソルバがあるときのモデルベースRL（TODO）

* [Is Plug-in Solver Sample-Efficient for Feature-based Reinforcement Learning?](https://arxiv.org/abs/2010.05673)
* [Minimax PAC bounds on the sample complexity of reinforcement learning with a generative model](https://link.springer.com/article/10.1007/s10994-013-5368-1)

この２つはモデルを推定＆その$\varepsilon$-最適解を返すソルバがある場合のモデルベースRLについて話してる．後で書こう．
